In [ ]:

#Install required libraries
import pandas as pd
from gen3.submission import Gen3Submission
from gen3.auth import Gen3Auth
from gen3.query import Gen3Query

In [ ]:
### download and import some custom Python scripts from https://github.com/cgmeyer/gen3sdk-python
# import os
#os.system("wget https://raw.githubusercontent.com/cgmeyer/gen3sdk-python/master/expansion/expansion.py")


--2025-02-26 18:17:44--  https://raw.githubusercontent.com/cgmeyer/gen3sdk-python/master/expansion/expansion.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 235477 (230K) [text/plain]
Saving to: ‘expansion.py.1’

     0K .......... .......... .......... .......... .......... 21% 12.1M 0s
    50K .......... .......... .......... .......... .......... 43% 11.0M 0s
   100K .......... .......... .......... .......... .......... 65% 18.7M 0s
   150K .......... .......... .......... .......... .......... 86% 21.5M 0s
   200K .......... .......... .........                       100% 12.2M=0.02s

2025-02-26 18:17:45 (14.3 MB/s) - ‘expansion.py.1’ saved [235477/235477]



0

In [39]:
from expansion import Gen3Expansion

In [43]:
# Initiate instances of the Gen3 SDK Classes using credentials file downloaded from https://staging.midrc.org/identity
# You can view the SDK code/functions in GitHub: https://github.com/uc-cdis/gen3sdk-python
api = 'https://qa-midrc.planx-pla.net/'
cred = 'midrc_api_key.json'
auth = Gen3Auth(api, refresh_file=cred) # authentication class
sub = Gen3Submission(api, auth) # submission class
query = Gen3Query(auth) # query class
exp = Gen3Expansion(api,auth,sub) # class with some custom scripts
exp.get_project_ids()

Getting all project_ids you have access to in https://qa-midrc.planx-pla.net/


Gen3AuthError: Failed to get an access token from https://qa-midrc.planx-pla.net/user/credentials/cdis/access_token:
<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
</body>
</html>


In [ ]:
img_stud = exp.get_node_tsvs(projects = 'jnkns-jenkins', node='imaging_study')

#increasing the volume of the number of records by 10 fold for testing purposes
img_stud_list = [img_stud.copy(deep=True) for i in range(10)]
for i in range(10):
    img_stud_list[i]["submitter_id"] = img_stud['submitter_id']+"_"+str(i) #changing the submitter_id to make them unique

img_stud = pd.concat(img_stud_list)


File previously downloaded.
node_tsvs/imaging_study_tsvs/jnkns-jenkins_imaging_study.tsv has 810 records.
length of all dfs: 810
Master node TSV with 810 total records written to master_imaging_study.tsv.


type
submitter_id	
datasets
cases

In [45]:
len(img_stud)

8100

In [ ]:
img_stud = img_stud[['type', 'submitter_id', 'datasets.submitter_id', 'cases.submitter_id']]

img_stud['datasets.submitter_id'] = 'dataset_backpedalled_usurers'


img_stud.to_csv('test_img_stud.tsv', sep='\t', index=False)



8100


In [37]:
import time
start = time.time()
sub_test = sub.submit_file(project_id = 'jnkns-jenkins', filename = 'test_img_stud.tsv', chunk_size=1000)
end = time.time()
time_taken = end - start



Submitting test_img_stud.tsv with 8100 records.
Chunk 1 (chunk size: 1000, submitted: 0 of 8100)
	 Succeeded: 1000 entities.
Chunk 2 (chunk size: 1000, submitted: 1000 of 8100)
	 Succeeded: 1000 entities.
Chunk 3 (chunk size: 1000, submitted: 2000 of 8100)
	 Succeeded: 1000 entities.
Chunk 4 (chunk size: 1000, submitted: 3000 of 8100)
	 Succeeded: 1000 entities.
Chunk 5 (chunk size: 1000, submitted: 4000 of 8100)
	 Succeeded: 1000 entities.
Chunk 6 (chunk size: 1000, submitted: 5000 of 8100)
	 Succeeded: 1000 entities.
Chunk 7 (chunk size: 1000, submitted: 6000 of 8100)
	 Succeeded: 1000 entities.
Chunk 8 (chunk size: 1000, submitted: 7000 of 8100)
	 Succeeded: 1000 entities.
Chunk 9 (chunk size: 1000, submitted: 8000 of 8100)
	 Succeeded: 100 entities.
Finished data submission.
Successful records: 8100
Failed invalid records: 0


In [ ]:

print(f"time taken to push 810 recoords with sheepdog load_test_branch - {time_taken}")

#These are the values of the time taken to push 8100 records to the service
time_load_test_branch = 628.9082880020142  #But the chunk size was cut down to 500
time_load_test_branch_new_base = 641.9752421379089 #but service didn't fail after 1min, keeping chunk size at 1000
time_load_2501 = 299.92830085754395 # Benchnark time for 8100 records with tag 25.01


time taken to push 810 recoords with sheepdog load_test_branch - 310.8567671775818
